In [1]:
import pygame
import numpy as np
import random
import os
import math
from screeninfo import get_monitors

# Constants
FIELD_OF_VIEW = 90.0
CHECKER_SIZE = 10.0 # Deg
BAR_WIDTH = 2 #Number of squares
MOVING_LIMITS = 45.0  # Deg
SIDE_DURATION = 3.0 # Seconds
SIDES_PER_PHASE = 4 #How many side motions should an object during the stimulus phase 
BETWEEN_PHASE_DURATION = 1.0 # Seconds
EXPERIMENT_DURATION = 20 # Number of loops
FPS = 60


# Initialize Pygame
monitors = get_monitors()
monitor = monitors[-1]
if len(monitors) > 1:
    os.environ['SDL_VIDEO_WINDOW_POS'] = "%d,%d" % (monitors[0].width, 0) #this will push the pygame window in the second monitor
    
WIDTH = monitor.width
HEIGHT = monitor.height
PIXEL_PER_DEG = WIDTH / FIELD_OF_VIEW
DEG_WIDTH = WIDTH / PIXEL_PER_DEG
DEG_HEIGHT = HEIGHT / PIXEL_PER_DEG
BAR_WIDTH = BAR_WIDTH * CHECKER_SIZE
PHASE_DURATION = SIDE_DURATION*SIDES_PER_PHASE
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

def generate_checkerboard(width, height):
    squares_per_row, squares_per_col = int(width / CHECKER_SIZE), int(height / CHECKER_SIZE)
    checkerboard = np.zeros((squares_per_row, squares_per_col), dtype=int)
    for i in range(squares_per_row):
        for j in range(squares_per_col):
            checkerboard[i, j] = random.choice([0, 1])  # 0 for black, 1 for white
    return checkerboard

def draw_rectangle(color, x, y, width, height):
    pygame.draw.rect(screen, color, (x*PIXEL_PER_DEG, y*PIXEL_PER_DEG, width*PIXEL_PER_DEG, height*PIXEL_PER_DEG))

def draw_checkerboard(pattern, offset_x, offset_y):
    for i in range(pattern.shape[0]):
        for j in range(pattern.shape[1]):
            color = WHITE if pattern[i, j] == 1 else BLACK
            draw_rectangle(color, (i * CHECKER_SIZE) + offset_x, (j * CHECKER_SIZE) + offset_y, CHECKER_SIZE, CHECKER_SIZE)

checkerboard = generate_checkerboard(DEG_WIDTH*2, DEG_HEIGHT+CHECKER_SIZE)
bar = generate_checkerboard(BAR_WIDTH, DEG_HEIGHT+CHECKER_SIZE)

#design stimuli (x is themoving object horizontal postion) 
PHASES = [
    lambda x: (
        screen.fill(WHITE),
        draw_rectangle(BLACK, x, 0, BAR_WIDTH, DEG_HEIGHT)
    ),
    lambda x: (
        draw_checkerboard(checkerboard, 0, 0),
        draw_checkerboard(bar, x, 0)
    ),
    lambda x: (
        draw_checkerboard(checkerboard, 0, 0),
        draw_rectangle(BLACK, x, 0, BAR_WIDTH, DEG_HEIGHT)
    ),
    lambda x: (
        draw_checkerboard(checkerboard, x, 0)
    )
]

LOOP_DURATION = (PHASE_DURATION+BETWEEN_PHASE_DURATION)*len(PHASES)
PHASE_OBECTS = [BAR_WIDTH, BAR_WIDTH, BAR_WIDTH, DEG_WIDTH*2]

pygame 2.5.1 (SDL 2.28.2, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

loop_number = 0
phase = 0
current_time = pygame.time.get_ticks()
loop_start_time = current_time
phase_start_time = current_time
object_size = PHASE_OBECTS[phase]
print("loop " + str(loop_number))
print("time " + str(current_time))

while loop_number <= EXPERIMENT_DURATION:

    quit = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            quit = True
    if quit:
        break
    current_time = pygame.time.get_ticks()
    loop_elapsed_time = (current_time - loop_start_time) / 1000.0  # Convert to seconds
    phase_elapsed_time = (current_time - phase_start_time) / 1000.0

    if loop_elapsed_time >= LOOP_DURATION:
        loop_number = loop_number +1
        phase = 0
        loop_start_time = current_time
        phase_start_time = current_time
        object_size = PHASE_OBECTS[phase]
        checkerboard = generate_checkerboard(DEG_WIDTH*2, DEG_HEIGHT+CHECKER_SIZE)
        bar = generate_checkerboard(BAR_WIDTH, DEG_HEIGHT+CHECKER_SIZE)
        print("loop " + str(loop_number))
        print("time " + str(current_time))

    elif phase_elapsed_time >= PHASE_DURATION + BETWEEN_PHASE_DURATION:
        phase += 1
        object_size = PHASE_OBECTS[phase]
        phase_start_time = current_time

    elif phase_elapsed_time >= BETWEEN_PHASE_DURATION:
        side_number = int(phase_elapsed_time/SIDE_DURATION)
        side_elpased_time = phase_elapsed_time-(side_number*SIDE_DURATION)
        is_side_even = -1 + (side_number%2 != 0)*2
        starting_pos = (DEG_WIDTH + (is_side_even*MOVING_LIMITS) - object_size)/2
        object_pos = starting_pos - (1-(SIDE_DURATION - side_elpased_time)/SIDE_DURATION)*MOVING_LIMITS*is_side_even
        PHASES[phase](object_pos)
    
    else:
        screen.fill(BLACK)

    
    pygame.display.update()
    clock.tick(FPS)
    
pygame.quit()

loop 0
time 267
